In [3]:
import pandas as pd
import json

In [7]:
def strip(line):
    # remove the first and last character
    return line

with open('../data/dblp.v12.json') as f:
    lines = f.readlines(100000000)
    data = [json.loads(line[1:-1]) for line in lines[2:]]

In [14]:
df = pd.DataFrame(data).dropna(subset=["references"]).reset_index(drop=True)
df

,id,authors,title,year,n_citation,page_start,page_end,doc_type,publisher,volume,issue,doi,fos,venue,references,indexed_abstract
0,1674,"[{'name': 'G. Beale', 'org': 'Archaeological C...",A methodology for the physically accurate visu...,2011,1,137,144,Conference,Eurographics Association,,,10.2312/VAST/VAST11/137-144,"[{'name': 'Statue', 'w': 0.40216}, {'name': 'E...",{'raw': 'International Conference on Virtual R...,"[1535888970, 1992876689, 1993710814, 203565334...","{'IndexLength': 201, 'InvertedIndex': {'illust..."
1,1688,"[{'name': 'Altaf Hossain', 'org': 'Department ...","Comparison of GARCH, Neural Network and Suppor...",2009,6,597,602,Conference,"Springer, Berlin, Heidelberg",,,10.1007/978-3-642-11164-8_97,[{'name': 'Autoregressive–moving-average model...,{'raw': 'Pattern Recognition and Machine Intel...,"[1560724230, 1986968751, 2156909104]","{'IndexLength': 64, 'InvertedIndex': {'standar..."
2,6522,"[{'name': 'Güzin Ulutas', 'org': 'Karadeniz Te...",Improved Secret Image Sharing Method By Encodi...,2011,0,331,335,Conference,"Springer, London",,,10.1007/978-1-4471-2155-8_42,"[{'name': 'Peak signal-to-noise ratio', 'w': 0...",{'raw': 'International Symposium on Computer a...,"[2007942710, 2040446335, 2042901705, 2138296484]","{'IndexLength': 89, 'InvertedIndex': {'44': [7..."
3,8373,"[{'name': 'Phan Cong Vinh', 'org': 'Department...",Formal agent-oriented ubiquitous computing: a ...,2012,0,498,508,Conference,"Springer, Berlin, Heidelberg",,,10.1007/978-3-642-28490-8_52,"[{'name': 'Ubiquitous commerce', 'w': 0.66293}...",{'raw': 'Asian Conference on Intelligent Infor...,"[141500162, 1977736041, 2057072261, 2495388083...","{'IndexLength': 83, 'InvertedIndex': {'before'..."
4,8763,"[{'name': 'Dominik Szajerman', 'org': 'Lodz Un...",Fur Visualisation for Computer Game Engines an...,2014,0,41,48,Conference,"Springer, Cham",,,10.1007/978-3-319-11331-9_6,"[{'name': 'Computer graphics (images)', 'w': 0...",{'raw': 'International Conference on Computer ...,"[2002239941, 2062162286, 2114709034]","{'IndexLength': 84, 'InvertedIndex': {'fur': [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30992,50952953,"[{'name': 'Satoshi Sato', 'org': 'Nagoya-unive...",A Person-Name Filter for Automatic Compilation...,2010,1,,,Conference,,,,,"[{'name': 'Bilingual person', 'w': 0.0}, {'nam...","{'raw': 'Language Resources and Evaluation', '...","[1571200267, 2144290838, 2163317568]","{'IndexLength': 170, 'InvertedIndex': {'Each':..."
30993,50955165,"[{'name': 'Narayan Gowraj', 'org': '""ANNA Univ...",S2MART: Smart Sql to Map-Reduce Translators,2013,5,571,582,Conference,"Springer, Berlin, Heidelberg",,,10.1007/978-3-642-37401-2_56,"[{'name': 'SQL', 'w': 0.64926}, {'name': 'Byte...","{'raw': 'Asia-Pacific Web Conference', 'id': 1...","[1493893823, 1573191158, 1993892970, 202543777...","{'IndexLength': 164, 'InvertedIndex': {'cluste..."
30994,50968780,"[{'name': 'Erhan Leblebici', 'org': 'Technisch...",A Comparison of Incremental Triple Graph Gramm...,2014,22,,,Journal,European Assoc. of Software Science and Techno...,67,,10.14279/tuj.eceasst.67.939,"[{'name': 'Graph', 'w': 0.0}, {'name': 'Model ...",{'raw': 'Electronic Communication of The Europ...,"[49859560, 1485542170, 1493088336, 1511832825,...","{'IndexLength': 129, 'InvertedIndex': {'but': ..."
30995,50968961,"[{'name': 'N. Sivakumar', 'org': 'Pondicherry ...",Testing Agent-Oriented Software by Measuring A...,2011,2,88,98,Conference,"Springer, Berlin, Heidelberg",,,10.1007/978-3-642-22714-1_10,"[{'name': 'Adaptability', 'w': 0.46004}, {'nam...",{'raw': 'Advances in Computing and Communicati...,"[1623971208, 1634504756, 1946080619, 2132850463]","{'IndexLength': 262, 'InvertedIndex': {'been':..."


In [50]:
df = (
    df[[
        "id", 
        "year", 
        "authors.name", 
        "author.org", 
        "venue.raw", 
        "keywords",
        "fos.name",
        "title", 
        "abstract",
        "n_citation",
        "references"
    ]]
    # .loc[df["language"] == "en"]
    .dropna()
)
df

KeyError: "['authors.name', 'author.org', 'venue.raw', 'keywords', 'fos.name', 'abstract'] not in index"

In [ ]:
df["text"] = (
    df["title"] + "\n" 
    + df["authors.name"] + "\n" 
    + df["author.org"] + "\n" 
    + df["venue.raw"] + "\n" 
    + df["fos.name"] + "\n" 
    + df["keywords"].apply(lambda x: " ".join(x)) + "\n"
    + df["abstract"]
)

In [19]:
from itertools import combinations

# get co-citation counts
co_cite_count = dict()
co_cite_ids = dict()
for row in df.itertuples():
    for combination in combinations(row.references, 2):
        c = frozenset(combination)
        if c in co_cite_count:
            co_cite_count[c] += 1
            co_cite_ids[c].append(row.id)
        else:
            co_cite_count[c] = 1
            co_cite_ids[c] = [row.id]